# CHAPTER 7
---
# Data Wrangling: Clean, Transform, Merge, Reshape

In [6]:
import pandas as pd
import numpy as np

## Combining and Merging Data Sets <font color='green'>[Essential]</font>
- pandas.merge connects rows in DataFrames based on one or more keys. This will
be familiar to users of SQL or other relational databases, as it implements database
join operations.
- pandas.concat glues or stacks together objects along an axis.

### Database-style DataFrame Merges <font color='green'>[Essential]</font>

In [7]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],'data2': range(3)})

In [8]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [9]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [10]:
# INNER guarda solamente las líneas definidas en los dos tablas
pd.merge(df1, df2, how='inner', on='key')  # el índice cambio !

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [11]:
# LEFT guarda solamente las líneas de la tabla izquierda df1
pd.merge(df1, df2, how='left', on='key')

,key,data1,data2
0,b,0,1.0
1,b,1,1.0
2,a,2,0.0
3,c,3,NaN
4,a,4,0.0
5,a,5,0.0
6,b,6,1.0


In [12]:
pd.merge(df1, df2, how='right', on='key')

,key,data1,data2
0,a,2.0,0
1,a,4.0,0
2,a,5.0,0
3,b,0.0,1
4,b,1.0,1
5,b,6.0,1
6,d,NaN,2


In [13]:
# OUTER guarda todas las líneas
pd.merge(df1, df2, how='outer', on='key')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


In [14]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 'data1': range(7)})

In [15]:
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],'data2': range(3)})

In [16]:
df3

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [17]:
df4

,rkey,data2
0,a,0
1,b,1
2,d,2


In [18]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [19]:
# suffixes
df5 = df4.copy()
df5['data1'] = 5

In [20]:
pd.merge(df3, df5, left_on='lkey', right_on='rkey')

,lkey,data1_x,rkey,data2,data1_y
0,b,0,b,1,5
1,b,1,b,1,5
2,b,6,b,1,5
3,a,2,a,0,5
4,a,4,a,0,5
5,a,5,a,0,5


In [21]:
pd.merge(df3, df5, left_on='lkey', right_on='rkey', suffixes=['_df3', '_df5'])

,lkey,data1_df3,rkey,data2,data1_df5
0,b,0,b,1,5
1,b,1,b,1,5
2,b,6,b,1,5
3,a,2,a,0,5
4,a,4,a,0,5
5,a,5,a,0,5


In [22]:
left = pd.DataFrame(
    {
        'key1': ['foo', 'foo', 'bar'],
        'key2': ['one', 'two', 'one'],
        'lval': [1, 2, 3]
    }
)
right = pd.DataFrame(
    {
        'key1': ['foo', 'foo', 'bar', 'bar'], 
        'key2': ['one', 'one', 'one', 'two'],
        'rval': [4, 5, 6, 7]
    }
)

In [23]:
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [24]:
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [25]:
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


### Concatenating Along an Axis <font color='green'>[Essential]</font>

In [28]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [29]:
# guardamos la información de la serie original con la palabra keys
pd.concat([s1, s2, s3], keys=['one', 'two', 'three'])

one    a    0
       b    1
two    c    2
       d    3
       e    4
three  f    5
       g    6
dtype: int64

In [30]:
# en concaténant selon l'axe des colonnes, on fait apparaître des trous
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'], sort=False) 

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [31]:
# Es la misma síntaxis par los DataFrame
arr = np.arange(12).reshape((3, 4))

In [32]:
df = pd.DataFrame(arr)

In [33]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11


In [34]:
pd.concat([df, df])

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11


In [35]:
pd.concat([df, df], axis=1)

,0,1,2,3,0,1,2,3
0,0,1,2,3,0,1,2,3
1,4,5,6,7,4,5,6,7
2,8,9,10,11,8,9,10,11


In [36]:
data = pd.DataFrame(
    np.arange(12).reshape((3, 4)),
    index=['Ohio', 'Colorado', 'New York'],
    columns=['one', 'two', 'three', 'four']
)

In [37]:
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [38]:
# usamos map en vez de apply para los índices !
data.index = data.index.map(str.upper) 
data.columns = data.columns.map(str.lower)

In [39]:
data

,one,two,three,four
OHIO,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


In [40]:
data.rename(index={'OHIO': 'INDIANA'}, columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11
